<a href="https://colab.research.google.com/github/epogrebnyak/rides-minimal/blob/master/rides_minimal_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pathlib
import json
import os
from dataclasses import dataclass
from typing import List, Tuple, Generator, Callable

import requests
import pandas as pd
import numpy as np

from scipy.spatial.distance import cdist

pd.set_option("mode.chained_assignment", None)

# Подготовка данных

## Функции для скачивания данных

In [0]:
import pathlib
import os
import zipfile

def make_filename(url):
  return pathlib.Path(url.split("/")[-1])

def get_from_web(url, file:pathlib.Path):
    r = requests.get(url)
    file.write_bytes(r.content)
    print("Downloaded", file, "from", url)

def download(url):
  file = make_filename(url)
  if file.exists():
    print("File already exists:", file)   
  else: 
    get_from_web(url, file)
  return str(file)

def unzip(file, folder):        
    print("Unpacking JSON files")
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(folder)
    print("Done unpacking JSON files to folder:", folder)


def download_and_unzip(url: str, destination_folder: str):
  path = download(url)
  if not os.path.exists(destination_folder):
    os.mkdir(destination_folder)
    unzip (path, destination_folder)
  else:
    print ("Folder already exists:", destination_folder)   
  return destination_folder

## Преобразование файлов JSON в CSV

In [0]:
def iterate(directory: str):
    for dict_ in yield_jsons(directory):
        for p in dict_["data"]:
            car = dict_["info"]["car_id"]
            ride = dict_["info"]["id"]
            yield dict(time=p[0], lat=p[2], lon=p[1], car=car, ride=ride)


def filenames(directory: str):
    return list(pathlib.Path(directory).glob("*.json"))


def yield_jsons(directory: str):
    for filename in filenames(directory):
        yield load_json(filename)


def load_json(filepath: str):
    """"Load a JSON file content"""
    with open(filepath, encoding="utf-8") as f:
        return json.load(f)


def summaries(folder: str):
    summary = lambda d: d["info"] 
    return pd.DataFrame(map(summary, yield_jsons(folder)))


def trackpoints(folder: str):
    return pd.DataFrame(iterate(folder))


def save(filename: str, getter: Callable, source_folder: str):
  if not os.path.exists(filename):
    print("Creating", filename)
    df = getter(source_folder)
    df.to_csv(filename, index=False)
    print("Saved", filename)
  print("File already exists:", filename)  

## Получить полный набор данных

In [0]:
RAW_DATA_URL = ("https://dl.dropboxusercontent.com" 
                "/sh/hibzl6fkzukltk9/AABTFmhvDvxyQdUaBsKl4h59a/"
                "data_samples_json.zip")

In [0]:
JSON_FOLDER="data" 
FULL_CSV="df_full.csv"
SUMMARY_CSV="summaries.csv"

In [9]:
download_and_unzip(url=RAW_DATA_URL, destination_folder=JSON_FOLDER)
save(FULL_CSV, trackpoints, source_folder=JSON_FOLDER)
save(SUMMARY_CSV, summaries, source_folder=JSON_FOLDER)

Downloaded data_samples_json.zip from https://dl.dropboxusercontent.com/sh/hibzl6fkzukltk9/AABTFmhvDvxyQdUaBsKl4h59a/data_samples_json.zip
Unpacking JSON files
Done unpacking JSON files to folder: data
Creating df_full.csv
Saved df_full.csv
File already exists: df_full.csv
Creating summaries.csv
Saved summaries.csv
File already exists: summaries.csv


В результате мы получаем два файла `df_full.csv` и `summaries.csv`, которые содержат необходимую информацию из исходных файлов в формате JSON, которые мы скачали по ссылке `RAW_DATA_URL`. Если файлы уже созданы в сессии, они пересоздаваться не будут.


# Функции для импорта данных

In [0]:
def get_date(df):
  dt = lambda x: str(pd.Timestamp(x, unit="s").date())
  return df.time.apply(dt)

def get_dataframe(filename, **kwargs):
  df_full = pd.read_csv(filename, 
            usecols=["car", "time", "lat", "lon"],
            **kwargs)
  df_full["date"] = get_date(df_full)
  return df_full[["car", "date", "time", "lat", "lon"]]

# Вспомогательные функции

## Комбинации из n по 2

In [0]:
from itertools import combinations

def get_combinations(n: int) -> List[Tuple[int, int]]:
    """Выбор комбинаций из n по 2."""
    return [(i, j) for i, j in combinations(range(n), 2)]

def count_combinations(n: int) -> int:
    return len(list(get_combinations(n)))

## Функции расстояний

In [0]:
from geopy.distance import great_circle

def distance_km(a: tuple, b: tuple):
    # great_circle() менее точен, но быстрее чем geopy.distance.distance
    # https://geopy.readthedocs.io/en/stable/#module-geopy.distance
    return great_circle(a, b).km

Coord = Tuple[float, float]

# используется с distance_delta
def safe_distance(a: Coord, b: Coord) -> float:
  #if not isinstance(b, tuple): # catch NaN
  #  return 0
  if a == b:
    return 0  
  try:  
    return distance_km(a, b)
  except ValueError:
    return np.nan
        

# используется с cdist
def safe_distance_2(a: Coord, b: Coord) -> float:    
    x1, y1 = a
    x2, y2 = b
    if np.isnan(x1) or np.isnan(x2):
        return np.nan
    else:
        return round(distance_km(a, b), 9)

## Кэширование длинных расчетов


In [0]:
def from_cache(filename: str, getter: Callable):
  """Прочитать объект из *filename* или
     создать объект с помощью getter и сохранить 
     его в *filename*.
     Возвращает значение getter().
  """
  import json, os  
  if os.path.exists(filename):
    with open(filename, "r") as f:
        return json.load(f)
  else:     
    content = getter()
    with open(filename, "w") as f:
        json.dump(content, f)
    return content

## Определить тип автомобиля 

In [0]:
def vehicle_type_dataframe(cars: pd.DataFrame):
    """Разобрать машины по типам:
       - bus        
       - passenger
       - freight
       - special
    """

    def has(string):
        return lambda s: string in s

    cars["type"] = "other"
    # bus
    cars.loc[cars.car_passengers >= 8, "type"] = "bus"
    # passenger
    ax = (cars.category == "Специальный\\Автобус ") & (cars.type == "other")
    bx = cars.category.apply(has("Легковой")) & (cars.type == "other")
    cars.loc[(ax | bx), "type"] = "passenger"
    # freight
    ix = cars.category.apply(has("Грузовой"))
    cars.loc[ix, "type"] = "freight"
    # special
    ax = cars.category.apply(has("Специальный")) & (cars.type == "other")
    bx = cars.category == "Строительный\\Автокран"
    cars.loc[(ax | bx), "type"] = "special"
    assert len(cars[cars.type == "other"].category.unique()) == 0
    return cars.type

In [0]:
def all_cars(source=SUMMARY_CSV):
  return (pd.read_csv(source)
    .groupby("car_id")
    .first())
  
VEHILCES = vehicle_type_dataframe(all_cars())

def vehilce_type(car_id: str):
  return VEHILCES.loc[car_id] 

assert 'freight' == vehilce_type("76727d5c-628d-4bcf-a24a-e5cf8a713426")

# Список треков

## Создание списка треков из датафрейма

In [0]:
@dataclass
class Trip:
    car: str
    date: str

class Route(pd.DataFrame):
    pass

def coord(df: pd.DataFrame):
    return list(zip(df.lat, df.lon))

def make_route(df: pd.DataFrame) -> Route:
    res = df[["time", "lat", "lon"]]
    res['coord'] = coord(df)
    return res.sort_values("time")

In [0]:
def get_trips_and_routes(df: pd.DataFrame) -> (List[Trip], List[Route]):
    """
    Преобразуем датафрейм в список поездок.
    """
    trips = []
    routes = []
    for date in df.date.unique():
        for car in df.car.unique():
            ix = (df.car == car) & (df.date == date)
            carday = df[ix]
            if not carday.empty:
              trips.append(Trip(car, str(date)))
              route = make_route(carday)
              routes.append(route)
    return trips, routes

def make_trips_dataframe(trips: List[Trip]):
    return pd.DataFrame(t.__dict__ for t in trips)  

## Работа с треком

In [0]:
def duration_acc(df: pd.DataFrame) -> pd.DataFrame:
    return (df.time - df.time.iloc[0]) / 60


def time_deltas(df: pd.DataFrame) -> pd.DataFrame:
    return df.time.diff()


def distance_deltas(df: pd.DataFrame) -> pd.DataFrame:
    res = df.copy()
    res["prev_coord"] = res.coord.shift(1)
    return res.apply(lambda x: safe_distance(x.coord, x.prev_coord), axis=1)


def milage_acc(df: pd.DataFrame) -> pd.DataFrame:
    return distance_deltas(df).cumsum().fillna(0)


def milage(route) -> float:
    return round(milage_acc(route).iloc[-1], 2)


def points(route_df: Route) -> np.ndarray:
    return np.array(route_df.coord.to_list())


def date(route):
    return route.iloc[0].time.date()


def timestamp(x):
    return pd.Timestamp(x, unit="s")


def nth_timestamp(route, n):
  return timestamp(route.iloc[n].time)


def start_time(route):
    return nth_timestamp(route, 0)
    

def end_time(route):
    return nth_timestamp(route, -1)

#  Функции апроксимации трека



## Задается количество точек апрокимации на трек

In [0]:
def true_quantile(xs, q):
  """Квантиль накопленной суммы чисел xs без учета повторов. 
     
     np.quantile дает неправильный для нас реузльтат:
     np.quantile(xs, 0.9) может быть равно len(xs),
     если в хвосте xs много повторов.
  """ 
  assert 0 <= q <= 1
  end_value = xs[-1] * q
  is_smaller = lambda x: 1 if x <= end_value else 0  
  return sum(map(is_smaller, xs))

def where(xs, q):
    i = true_quantile(xs, q)    
    return np.searchsorted(xs, xs[i], side="left")

def find_index_one(xs: List, q: float) -> int:
    """Перестраховываемся по сравнению с where.""" 
    if q == 0:
        return 0
    if q == 1:
        return -1
    return where(xs, q)    

def find_index(xs: List, n_segments: int) -> List[int]:
    qs = np.linspace(0, 1, n_segments+1)
    return [find_index_one(xs, q) for q in qs]

def n_segments_by(n: int, route: Route, acc_with: Callable):
    xs = acc_with(route).tolist()
    ix = find_index(xs, n)
    return route.iloc[ix]

def n_segments_by_distance_(n: int, route: Route):
    return n_segments_by(n, route, milage_acc)

def n_segments_by_distance(n: int):
    return lambda route: n_segments_by_distance_(n, route)

def n_segments_by_time_(n: int, route: Route):
    return n_segments_by(n, route, duration_acc)

def n_segments_by_time(n: int):
    return lambda route: n_segments_by_time_(n, route)


#r = pd.read_csv("r1.csv")
#xs = [x for x in milage_acc(r)]
#linspace(10), find_index(xs, 10),
#true_quantile(xs, .2)    
#find_index_one(xs, 1)

#r.sample(10).coord#distance_deltas()
#a = (54.644335999999996, 22.734314)
#b = (54.59284, 22.178588)
#distance_km(a,b)

## Задается шаг апрокисмации трека (в км или минутах)

In [0]:
def growing_index(xs, step):
    xs = [x for x in xs]
    result = [0]  # will include start point
    current = xs[0]
    for i, x in enumerate(xs):
        accumulated = x - current
        if accumulated >= step:
            result.append(i)
            accumulated = 0
            current = x
    n = len(xs) - 1
    if result[-1] != n: # will include end point
        result.append(n)
    return result

def time_increment_(minutes: int, route: Route):
    xs = duration_acc(route)
    ix = growing_index(xs, minutes)
    return route.iloc[ix]


def time_increment(minutes):
    return lambda route: time_increment_(minutes, route)


def distance_increment_(step_km: float, route: Route):
    xs = milage_acc(route)
    ix = growing_index(xs, step_km)
    return route.iloc[ix]


def distance_increment(step_km: float):
    return lambda route: distance_increment_(step_km, route)

## Другие апроксимации

- [ ] по астрономическому времени (есть в ноутбке `rides2`)
- [ ] по остановкам (есть в "тяжелом" алгоритме)

# Поиск сходимости

## Матрица расстояний между точками двух треков



In [0]:
def proximity(r1, r2):
    mat = cdist(points(r1), points(r2), safe_distance_2)
    return Proximity(mat)

@dataclass
class Proximity:
    """Матрица расстояний между двумя треками."""

    mat: np.ndarray

    def min(self):
        """
    Минимиальное расстояние между всеми точками двух треков.
    """
        return round(np.nanmin(self.mat), 3)

    def max(self):
        """
    Максимальное расстояние между всеми точками двух треков.
    """
        return round(np.amax(self.mat), 3)

    def side_min(self, axis: int):
        # 1 is to search by columns - applies to searching minima for 1st route
        # 0 is to search by rows - applies to searching minima for 2nd route
        x = np.nanmin(self.mat, axis)
        return x[~np.isnan(x)]

    def coverages(self):
        return Coverage(self.side_min(1)), Coverage(self.side_min(0))

    def report(self, search_radius: float):
      c1, c2 = self.coverages()
      return dict(
                min_dist=self.min(),
                max_dist=self.max(),
                cov_1=c1.coverage(search_radius),
                cov_2=c2.coverage(search_radius),
            )    


@dataclass
class Coverage:
    """Для каждой точки выбранного трека посчитано 
       минимальное из расстояний до точек другого трека.
       
       Если в треке 10 точек, то минимальных расстояний 
       до другого трека будет 10.
       
       Мы выбираем те из расстояний, которые меньше заданного 
       порога(радиуса) сближения.
              
       Предположим в треке 10 точек и минимальные расстояния до 
       фигуры второго трека такие:
       
       >> mins = [21.2, 22.6, 17.6, 7.7, 2.7, 0.3, 0.4, 0.6, 0.2, 3.1]
       
       Тогда "сблизившимися" с другим треков при радиусе точности 1 
       мы считаем 4 точки:
       
       >> radius = 1           
       >> [x for x in mins if x < radius]
       [0.3, 0.4, 0.6, 0.2]
       
       Коэффциент перекрытия составляет 4/10 = 40%. 
       
       Он показывает, что 40% точек данного трека
       находились на расстоянии не более чем *radius*
       от какой-то точки другого трека.
       
       Замечания:
       
       - Коэффциент перекрытия (КП) может интепретироваться как 
         часть пути, если отрезки между точками равные. 
         
       - Значение коэффциента перекрытия зависит от точности 
         апроксимации трека и радиуса.       
         
       - Треки могут иметь сложную форму, высокий КП не гарантирует,
         что доставку по одному треку можно переложить на другой трек.
         Но при низких КП это заведомо невозможно.
         
       - Нулевой КП - машины ездили в разных местах.  
         
    """

    mins: np.ndarray

    def in_proximity(self, radius: float):
        """
           Количество точек с минимальными расстояниями до другого трека.
           Минималаьное расстояние - это расстояние, величина которого
           меньше заданного радиуса сближения.
        """
        return (self.mins < radius).sum() #self.mins[self.mins < radius]

    def coverage(self, radius: float):
        """
           Коэффициент перекрытия - доля трека, которая находится 
           на расстоянии не более заданного радиуса от другого трека.
        """
        return round(self.in_proximity(radius) / len(self.mins), 2)

## Функции поиска сходимости





In [0]:
def inspect(routes, simplify_with=distance_increment):
  def pair(i, j, search_radius, *arg, **kwarg):
      f = simplify_with(*arg, **kwarg)
      r1 = f(routes[i])
      r2 = f(routes[j])
      return proximity(r1, r2).report(search_radius)
  return pair

def search(
    routes: List[Route],
    simplify_with: Callable,
    search_radius_1: float,
    refine_with = Callable,
    search_radius_2 = float,
    limit=None,
):

    # Урезаем для демо-примеров
    if limit:
        routes = routes[0:limit]
        print(f"Trimmed dataset to {limit} pairs")

    m = len(routes)
    print(f"{count_combinations(m)} pairs are possible for {m} routes")

    # Грубая апроксимация треков    
    print(f"Simplified {m} routes")
    rough_routes = [simplify_with(route) for route in routes]

    def prox(i, j):  # замыкание для удобного доступа к routes
        return proximity(rough_routes[i], rough_routes[j])

    # Выбор пересекающися пар
    pairs = [(i,j) for (i,j) in get_combinations(m) 
              if prox(i,j).min() < search_radius_1]
    print(f"Found {len(pairs)} pairs of intersecting routes")

    # Уточняем апроксимацию треков   
    def members(pairs):
      from itertools import chain
      return set(chain.from_iterable(pairs))

    refined_routes = list(range(m)) 
    for k, i in enumerate(members(pairs)):
        refined_routes[i] = refine_with(routes[i])
    print(f"Made better approximation of {k} routes")
    print("Calculating distances between routes and reporting...")

    # Считаем перекрытие треков в пересекающися парах
    from tqdm import tqdm
    result = []
    for (i, j) in tqdm(pairs):
        p = proximity(refined_routes[i], refined_routes[j])
        if p.min() < search_radius_1:
            d = dict(
                id_1=i, 
                id_2=j,
                **p.report(search_radius_2)
            )
            result.append(d)
    return result

# Пример расчетов

### Получаем данные

In [0]:
# Получаем полный массив исходных данных
try:
  df_full
except NameError:    
  df_full = get_dataframe(FULL_CSV)

In [24]:
days = ['2019-09-09']
types = ['freight']

# 1. Создаем выборку треков по дням и типам машин и упрощаем ее 
def subset(df, days, types):
  ix = df_full.date.isin(days)
  res = df_full[ix]
  kx = res.car.apply(vehilce_type).isin(types)
  return res[kx]
subset_df = subset(df_full, days, types)

# 2. Преобразуем датафрейм в список поедок
trips, routes = get_trips_and_routes(subset_df)
len(trips)  

46

In [25]:
# Пробег по поездкам
milages = [milage(r) for r in routes]

/usr/local/lib/python3.6/dist-packages/geopy/point.py:180: UserWarning: A single number has been passed to the Point constructor. This is probably a mistake, because constructing a Point with just a latitude seems senseless. If this is exactly what was meant, then pass the zero longitude explicitly to get rid of this warning.
  UserWarning)


In [0]:
gen = [dict(car=t.car, 
            start=start_time(r),
            end=end_time(r),
            km=m) 
       for (t, r, m) in zip(trips, routes, milages)]
pd.DataFrame(gen).to_csv("trips.csv")

## ДеБАГ

In [0]:
r = routes[1]


In [28]:
for i, r in enumerate(routes):
  print(i)
  n_segments_by_distance(n=10)(r)

0


/usr/local/lib/python3.6/dist-packages/geopy/point.py:180: UserWarning: A single number has been passed to the Point constructor. This is probably a mistake, because constructing a Point with just a latitude seems senseless. If this is exactly what was meant, then pass the zero longitude explicitly to get rid of this warning.
  UserWarning)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


### Поиск близких пар

In [29]:
# Расчет попарных характеристик поездок

def default_search(routes: List[Route], limit=None):
   return search( 
    routes,
    simplify_with = n_segments_by_distance(n=10),
    search_radius_1 = 10,
    refine_with = distance_increment(step_km=2.5),
    search_radius_2 = 2.5 * 1.2,
    limit=limit,
)   
result_dicts = default_search(routes, limit=None)

1035 pairs are possible for 46 routes
Simplified 46 routes


/usr/local/lib/python3.6/dist-packages/geopy/point.py:180: UserWarning: A single number has been passed to the Point constructor. This is probably a mistake, because constructing a Point with just a latitude seems senseless. If this is exactly what was meant, then pass the zero longitude explicitly to get rid of this warning.
  UserWarning)


Found 488 pairs of intersecting routes


  0%|          | 1/488 [00:00<01:27,  5.59it/s]

Made better approximation of 45 routes
Calculating distances between routes and reporting...


100%|██████████| 488/488 [00:37<00:00, 12.88it/s]


In [30]:
# Формируем выгрузку данных в csv
  
def overlap(df):
    return ((df.cov_1*df.len_1 + df.cov_2*df.len_2)
               .divide(df.len_1+df.len_2)
               .round(2)
               )

def add_vars(df, km = lambda i: milages[i]):
    return (df.assign(cov=lambda df: df.cov_1 + df.cov_2)
              .assign(len_1=lambda df: df.id_1.apply(km))
              .assign(len_2=lambda df: df.id_2.apply(km)) 
              .assign(op=overlap)
      )

result_df = (
      add_vars(pd.DataFrame(result_dicts))
      .sort_values("cov", ascending=False)
      .head(20)
      .reset_index(drop=True)
  )  
result_df.to_csv("output.csv", index=None)
result_df

,id_1,id_2,min_dist,max_dist,cov_1,cov_2,cov,len_1,len_2,op
0,16,20,0.003,2.785,1.00,1.00,2.00,28.95,6.73,1.00
1,13,20,0.023,4.869,1.00,1.00,2.00,30.74,6.73,1.00
2,23,32,0.012,59.173,1.00,1.00,2.00,136.51,71.29,1.00
3,13,16,0.026,4.944,1.00,1.00,2.00,30.74,28.95,1.00
4,8,43,0.004,35.133,1.00,1.00,2.00,138.40,110.82,1.00
5,9,13,0.007,5.622,1.00,1.00,2.00,54.92,30.74,1.00
6,9,16,0.036,4.300,1.00,1.00,2.00,54.92,28.95,1.00
7,11,18,0.282,11.183,1.00,1.00,2.00,27.56,53.84,1.00
8,0,14,0.004,87.231,0.96,1.00,1.96,524.50,388.86,0.98
9,9,20,0.034,4.296,0.96,1.00,1.96,54.92,6.73,0.96


# Графические функции

In [0]:
"""Графические функции для отрисовки треков"""

import matplotlib.pyplot as plt
from matplotlib import cm


def get_color(i, n):
    if n <= 10:
        cmap = cm.get_cmap(name="tab10")
        return cmap(i)
    else:
        cmap = cm.get_cmap(name="jet")
        return cmap(i / n)


def yield_route_and_color(routes):
    n = len(routes)
    for i, r in enumerate(routes):
        col = get_color(i, n)
        yield r, col


def make_plotter(axis_plotter_func):
    def plotter(routes, ax=None, title=""):
        if ax is None:
            f, ax = plt.subplots()
        n = len(routes)
        for i, r in enumerate(routes):
            col = get_color(i, n)
            axis_plotter_func(ax, r, col)
        ax.set_title(title)
        return ax

    return plotter


def scatter(ax, route, col):
    ax.scatter(x=route.lon, y=route.lat, s=0.5, alpha=0.8, color=col)


def segments(ax, route, col):
    ax.plot(route.lon, route.lat, lw=1, alpha=0.8, linestyle=":", color=col)


plot_points = make_plotter(scatter)
plot_connections = make_plotter(segments)


def plot_points_and_connections(routes, title="", ax=None):
    ax = plot_points(routes, ax)
    plot_connections(routes, ax)


def plot_raw_and_reduced(f1, f2, r1, r2, title="Общий заголовок"):
    fig, (ax1, ax2) = plt.subplots(
        nrows=1, ncols=2, figsize=(10, 4), sharex=True, sharey=True
    )
    plot_points_and_connections([f1, f2], ax=ax1, title="Исходные треки")
    plot_points_and_connections([r1, r2], ax=ax2, title="Упрощенные треки")
    plt.suptitle(title)


def plot_two(routes, i: int, j: int, simplify_with=None, title=""):
    """Отрисовка пары маршрутов по индексам в выборке."""
    f1, f2 = routes[i], routes[j]
    if simplify_with:
        r1, r2 = simplify_with(f1), simplify_with(f2)
    else:
        r1, r2 = f1, f2
    title = f"Поездки {i} и {j}"
    plot_raw_and_reduced(f1, f2, r1, r2, title)


def plot_one(route):      
    return plot_points([route])
            

## Пример

In [32]:
plot_two(routes, 32, 46, distance_increment(1))
milages[32], milages[46]

IndexError: ignored